<a href="https://colab.research.google.com/github/JamesExeter/WebScrapingAmazonData-Yucky/blob/main/WebScrapingAmazonData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time 
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
  

In [10]:
no_pages = 2

def get_data(pageNo):
  headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

  r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies = proxies
  content = r.content
  soup = BeautifulSoup(content)
  #print(soup)

  alls = []
  for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
    # print(d)
    name = d.find('span', attrs = {'class':'zg-text-center-align'})
    n = name.find_all('img', alt=True)
    # print(n[0]['alt'])

    author = d.find('a', attrs = {'class': 'a-size-small a-link-child'})
    rating = d.find('span', attrs = {'class':'a-icon-alt'})
    users_rated = d.find('a', attrs = {'class':'a-size-small a-link-normal'})
    price = d.find('span', attrs = {'class': 'p13n-sc-price'})

    all1 = []

    if name is not None:
      # print(n[0]['alt'])
      all1.append(n[0]['alt'])
    else:
      all1.append('unknown-product')

    if author is not None:
      # print(author.text)
      all1.append(author.text)
    elif author is None:
      author = d.find('span', attrs = {'class': 'a-size-small a-color-base'})
      if author is not None:
        all1.append(author.text)
      else:
        all1.append('0')
    
    if rating is not None:
      #print(rating.text)
      all1.append(rating.text)
    else:
      all1.append('-1')

    if users_rated is not None:
      #print(users_rated.text)
      all1.append(users_rated.text)
    else:
      all1.append('0')

    if price is not None:
      #prnt(price.text)
      all1.append(price.text)
    else:
      all1.append('0')

    alls.append(all1)

  return alls

In [30]:
results = []

for i in range(1, no_pages+1):
  results.append(get_data(i))

flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results), columns = ['Book Name', 'Author', 'Rating', 'Customers Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [31]:
df = pd.read_csv('amazon_products.csv')

df.shape
df.head()

,Book Name,Author,Rating,Customers Rated,Price
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"7,989",₹ 324.00
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"12,994",₹ 425.00
2,The Alchemist,Paulo Coelho,4.6 out of 5 stars,"42,861",₹ 175.00
3,Quantitative Aptitude for Competitive Examinat...,R S Aggarwal,4.4 out of 5 stars,"7,362",₹ 465.00
4,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.6 out of 5 stars,"26,766",₹ 424.00


In [32]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0]) 
df['Rating'] = pd.to_numeric(df['Rating'])

df['Price'] = df['Price'].str.replace('₹', '')
df['Price'] = df['Price'].str.replace(',', '')
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])
df['Price'] = df['Price'].astype(int)

df['Customers Rated'] = df['Customers Rated'].str.replace(',', '')
df['Customers Rated'] = pd.to_numeric(df['Customers Rated'], errors='ignore')
df.head()
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers Rated      int64
Price                int64
dtype: object

In [33]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan)

count_nan = len(df) - df.count()
count_nan

df = df.dropna()

In [34]:
data = df.sort_values(['Price'], axis=0, ascending=False)[:15]
data

,Book Name,Author,Rating,Customers Rated,Price
92,B D Chaurasia's Human Anatomy 4 Volume Set ( V...,Chaurasia,4.6,282,1664
58,My First Complete Learning Library: Boxset of ...,Wonder House Books,4.5,2492,799
19,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,4654,698
91,Objective NCERT at your FINGERTIPS for NEET-AI...,MTG Editorial Board,4.7,337,678
99,How to Prepare for Quantitative Aptitude for t...,Arun Sharma,4.5,1603,640
36,Oswaal CBSE SAMPLE QUESTION PAPERS CLASS 10 (S...,Oswaal Editorial Board,3.9,24,570
24,A Modern Approach to Verbal & Non-Verbal Reaso...,R.S. Aggarwal,4.4,3127,549
30,Unfinished: A Memoir,Priyanka Chopra Jonas,-1.0,0,531
15,Atomic Habits: The life-changing million copy ...,James Clear,4.6,11603,525
44,Kiran SSC Mathematics Chapterwise And Typewise...,Think Tank of Kiran Institute of,4.2,120,499


In [35]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()

In [36]:
p = figure(x_range = data.iloc[:,1], plot_width=800, plot_height=500, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color= None
p.y_range.start = 0

p.axis.major_label_orientation = math.pi/2

show(p)

In [37]:
data = df[df['Customers Rated'] > 1000]
data = df.sort_values(['Rating'], axis=0, ascending=False)[:15]
data

,Book Name,Author,Rating,Customers Rated,Price
34,Bhagavad Gita: Yatharoop (Hindi),A.C. Bhaktivendanta Swami Prabhupada,4.8,4131,179
70,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,21356,310
12,Think Like a Monk: The secret of how to harnes...,Jay Shetty,4.7,7449,362
45,Oswaal CBSE Question Bank Class 10 Social Scie...,Oswaal Editorial Board,4.7,192,261
40,What I Love About You and Our Memories: A Fill...,Sanjay D,4.7,150,259
55,Oswaal CBSE Question Bank Class 10 Mathematics...,Oswaal Editorial Board,4.7,191,265
63,The Last Queen,Divakaruni Chitra Banerjee,4.7,13,457
29,Oswaal CBSE Question Bank Class 10 Science (Re...,Oswaal Editorial Board,4.7,231,299
67,The Magic of the Lost Temple,Sudha Murty,4.7,1561,158
80,Diary of a Wimpy Kid: The Deep End (Book 15),Jeff Kinney,4.7,3718,375


In [52]:
p = figure(x_range=data.iloc[:,0], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,0], top=data.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

show(p)

In [53]:
data = df.sort_values(['Customers Rated'], axis=0, ascending=False)[:20]
data

,Book Name,Author,Rating,Customers Rated,Price
2,The Alchemist,Paulo Coelho,4.6,42861,175
72,The Subtle Art of Not Giving a F*ck: A Counter...,Mark Manson,4.4,37939,349
41,Think and Grow Rich,Napoleon Hill,4.5,36844,165
10,Rich Dad Poor Dad: What the Rich Teach Their K...,Robert T. Kiyosaki,4.6,35981,367
4,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.6,26766,424
38,Man's Search For Meaning: The classic tribute ...,Viktor E Frankl,4.5,23684,174
25,The Power of Your Subconscious Mind,Joseph Murphy,4.5,23548,159
76,The Power of your Subconscious Mind,Joseph Murphy,4.5,23548,156
70,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,21356,310
21,The Intelligent Investor (English) Paperback –...,Benjamin Graham,4.4,16752,482


In [55]:
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.palettes import d3
#colors has a list of colors which can be used in plots
colors = itertools.cycle(palette)

palette = d3['Category20'][20]

index_cmap = factor_cmap('Author', palette=palette, factors=data['Author'])

In [56]:
p = figure(plot_width=700, plot_height=700, title = "Top Author: Rating vs Customers Rated")
p.scatter('Rating', 'Customers Rated', source=data, fill_alpha=0.6, fill_color=index_cmap, size=20, legend='Author')
p.xaxis.axis_label = 'RATING'
p.yaxis.axis_label = 'CUSTOMERS RATED'
p.legend.location = 'top_left'

show(p)